# Data Exploration

This notebook is meant to be a template for exploration of data collected from trials.

## Prior to running the notebook

### Start the necessary processes
The model registry and orchestrator process must be started prior to running the notebook. To start the processes:
  1. Open a terminal
  2. Activate the `.venv` environment
  3. Run:
      ```
      python -m launch_local_services
      ```

Once you are done with working in this notebook, close the terminal process to stop the cogment services.

### Configure local datastore

By default, experiments do not persist the trial data to disk. To persist the data to disk, go to your experiment configuration and add the following line to the default list:
```
  - override /services/trial_datastore: local
```

## Cogment Setup

Necessary for the notebook to communicate with Cogment services. Without this cell, the imports from cogment_verse will fail.

In [ ]:
import os
import sys

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), "../"))
sys.path.append(ROOT_DIR)

from cogment_verse.utils.generate import generate
from cogment_verse.app import SPEC_FILEPATH

WORK_DIR = os.path.join(ROOT_DIR, ".cogment_verse")

generate(WORK_DIR, SPEC_FILEPATH)

TRIAL_DATASTORE_ENDPOINT = "grpc://localhost:9001"

## Imports

Install the necessary dependencies and libraries in your Python environment

In [ ]:
from utils import get_trial_data, get_trial_ids

from cogment_verse.specs import PLAYER_ACTOR_CLASS, WEB_ACTOR_NAME, EnvironmentSpecs

## Extract trial ids from datastore

In [ ]:
trial_ids = await get_trial_ids(TRIAL_DATASTORE_ENDPOINT)
print(f"Trial ids count: {len(trial_ids)}")
print(f"first 10 trial ids: [{', '.join(trial_ids[:10])}]")

## Extract Trial Data

Example extracting trial info and `samples` for all trials in the datastore.

If only interested in a single or specific set of trial ids, simply pass elements in the list argument `trial_ids` of the function `get_trial_data`.

In [ ]:
trial_data = await get_trial_data(TRIAL_DATASTORE_ENDPOINT, trial_ids=[])

To retrieve the info and samples from a specific trial, change the value of `test_trial_id` to an actual trial id.

In [ ]:
# **** Change this trial id for an id returned by the cell named: Extract trial ids from datasore
test_trial_id = "example_trial_id_0_0"

# Trial info about a specific trial ID
info = trial_data[test_trial_id]["trial_info"]

# List of samples for specific Trial ID
samples = trial_data[test_trial_id]["samples"]
print(f"Number of samples in trial: {len(samples)}")

## Inspect Trial Data

Each trial comes with a set of generic information about the trial. It is an instance of `TrialInfo` class. 

For more information about the `TrialInfo` class, see https://cogment.ai/docs/reference/python#class-trialinfo. 

In [ ]:
print(f"Trial ID: {info.trial_id}")
print(f"Trial State: {info.trial_state}")
print(f"Number of steps: {info.sample_count}")

print(f"Environment Implementation: {print(info.parameters.environment_implementation)}")
print(f"Environment Config: {info.parameters.environment_config}")

## Inspect Sample

Each sample from the trial datastore is an instance of the `DatastoreSample` class. 

For more info on the `DatastoreSample` class, see: https://cogment.ai/docs/reference/python#class-datastoresample




In [ ]:
sample = samples[0]
print(sample)

environment_specs = EnvironmentSpecs.deserialize(info.parameters.actors[0].config.environment_specs)
action_space = environment_specs.get_action_space()
observation_space = environment_specs.get_observation_space()

# General Sample info
print(f"Trial ID: {sample.trial_id}")
print(f"Trial State: {sample.trial_state}")
print(f"Step: {sample.tick_id}")
print(f"Timestamp: {sample.timestamp}")

# Actors
actor_names = list(sample.actors_data.keys())
test_actor_name = WEB_ACTOR_NAME if WEB_ACTOR_NAME in actor_names else PLAYER_ACTOR_CLASS

print(f"Actors: {actor_names}")
print(f"Selected samples from actor: {test_actor_name}")

# Observation, action, reward
cogment_obs = sample.actors_data[test_actor_name].observation
cogment_action = sample.actors_data[test_actor_name].action

# Deserialize observation and action
reward = sample.actors_data[test_actor_name].reward
observation = observation_space.deserialize(cogment_obs).value
action = action_space.deserialize(cogment_action).value

print(f"Observation: {observation}")
print(f"Action: {action}")
print(f"Reward: {reward}")